

      As a result of having a bad GPU at the moment, the framework used was Google Colab. The model used is Keras Convolutional Neural Network (CNN). This model classifies between 4 types of image orientations i.e. 0, 90, 180 and 270 degrees. With having around 329 Images in the training dataset and 112 for the validation dataset, we are able to achieve 94.43% after a series of constant training. 

**Loading Google drive to store files and retrieve if necessary.**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'proj/'

**Importing the necessary libraries required**

In [0]:
from keras.models import load_model
from keras.models import Sequential 
from keras.layers import Convolution2D, Activation, Dropout
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from IPython.display import display
from sklearn.metrics import classification_report, confusion_matrix
from PIL import Image
import numpy as np

**Allows user to upload datasets(.zip)**

In [0]:
from google.colab import files
uploaded = files.upload()
uploaded1 = files.upload()

**Creates a folder called dataset and unzips both the train_data.zip and validation_data.zip into the folder.**


In [0]:
!mkdir dataset
!unzip train_data.zip -d dataset/
!unzip validation_data.zip -d dataset/

**Initialising variables used in the program**

In [0]:
batch_size = 32
epochs = 50
num_of_train_samples = 329
num_of_test_samples = 112

**Keras model's main architecture**

In [0]:
model = Sequential()

model.add(Convolution2D(32, (3,3), input_shape=(64,64,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Convolution2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(output_dim=64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=4, activation='softmax'))

model.summary()

**Compiling the model**

In [0]:
model.compile(optimizer ='adam', loss ='categorical_crossentropy', metrics=['accuracy'])

**Train_data generator and test_data generator are data augmentors to create more images and preprocess them in order to improve the model's accuracy in respect to variation.**

**Training_set and test_set datasets are being preprocessed and normalised by dimensions of 64x64 in order to prevent errors and other factors from affecting the model.**

In [0]:
train_datagen = ImageDataGenerator(
          rescale=1./255,
          shear_range=0.2,
          zoom_range=0.2,
          horizontal_flip=False)

test_datagen = ImageDataGenerator(rescale=1./255)


training_set = train_datagen.flow_from_directory(
                directory='dataset/train_data',
                target_size=(64, 64),
                batch_size=32,
                shuffle=False,
                class_mode='categorical')

test_set= test_datagen.flow_from_directory(
                directory='dataset/validation_data',
                target_size=(64, 64),
                batch_size=32,
                shuffle=False,
                class_mode='categorical')

**Model begins training of 50 epochs**

In [0]:
model.fit_generator(
           training_set,
           steps_per_epoch=num_of_train_samples // batch_size,
           epochs=50,
           validation_data=test_set,
           validation_steps=num_of_test_samples // batch_size)

**Calculation of the test_steps_per_epoch variable**

In [0]:
test_steps_per_epoch = np.math.ceil(test_set.samples / batch_size)

**Resets the test_set dataset in order to prevent rearranging that's caused by the model, where the prediction is found by the predict_generator function on the test_set data. predicted_classes and class_labels are to obtain the classes in the dataset and the ground truth. **

**Prints a confusion matrix to visualise the model's classifications and mistakes as well as outputs a classification report of the model.**

In [0]:
test_set.reset()
predictions = model.predict_generator(test_set, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_set.classes
class_labels = list(test_set.class_indices.keys()) 
print('Confusion Matrix')
print(confusion_matrix(true_classes, predicted_classes))
print('Classification Report')  
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)  

**Saves the model and weights of the Keras CNN as .h5 files to export in the API.**

In [0]:
#model.save(root_dir + " myproj_model.h5")
#model.save_weights(root_dir + "my_model_weights.h5")